<a href="https://colab.research.google.com/github/kjahan/learning_to_rank/blob/develop/metrics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Download MSLR dataset

https://github.com/lezzhov/learning_to_rank/tree/main/learning_to_rank/data

In [1]:
!mkdir data
!wget -P /content/data https://raw.githubusercontent.com/lezzhov/learning_to_rank/refs/heads/main/learning_to_rank/data/test.txt
!wget -P /content/data https://raw.githubusercontent.com/lezzhov/learning_to_rank/refs/heads/main/learning_to_rank/data/train.txt
!wget -P /content/data https://raw.githubusercontent.com/lezzhov/learning_to_rank/refs/heads/main/learning_to_rank/data/vali.txt

--2024-12-02 02:29:15--  https://raw.githubusercontent.com/lezzhov/learning_to_rank/refs/heads/main/learning_to_rank/data/test.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 31307350 (30M) [text/plain]
Saving to: ‘/content/data/test.txt’

test.txt            100%[===================>]  29.86M  27.9MB/s    in 1.1s    

2024-12-02 02:29:17 (27.9 MB/s) - ‘/content/data/test.txt’ saved [31307350/31307350]

--2024-12-02 02:29:17--  https://raw.githubusercontent.com/lezzhov/learning_to_rank/refs/heads/main/learning_to_rank/data/train.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.


In [2]:
!ls -l data

total 93996
-rw-r--r-- 1 root root 31307350 Dec  2 02:29 test.txt
-rw-r--r-- 1 root root 34322657 Dec  2 02:29 train.txt
-rw-r--r-- 1 root root 30607789 Dec  2 02:29 vali.txt


## Evaluating the Models

In order to evaluate the performance of any model we need a distance metric. In this notebook we work through some commonly used distance metrics for evaluating learning to rank models. This notebook covers:

1. mean squared error (MSE)
2. pairwise difference (PD)
3. Discounted Cumulative Gain (DCG)
4. Normalised Discounted Cumulative Gain (nDCG)

We start by loading in the data and fitting a basic linear regression model which we then use to make predictions for the test set.

https://github.com/kjahan/learning_to_rank/blob/develop/02-metrics.ipynb

In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import sklearn
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [5]:
def extract_qid(qid_str):
    return qid_str[4:]

def extract_val(feat):
    return feat.split(':')[1]


def df_transform(df):
    df[1] = df[1].apply(extract_qid)
    df[df.columns[2:]] = df[df.columns[2:]].applymap(extract_val)
    return df

In [16]:
train = pd.read_csv("/content/data/train.txt", header=None, sep=" ")

# Drop last column
train = train.drop(138, axis=1)

# Drop rows with any NaN values
train.dropna(inplace=True)

print(train.shape)

# Count total NaNs in the DataFrame
total_nans = train.isna().sum().sum()
print(total_nans)

train_df = df_transform(train)

(29648, 138)
0


<ipython-input-5-54b665ed11d8>:10: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df[df.columns[2:]] = df[df.columns[2:]].applymap(extract_val)


In [11]:
test = pd.read_csv("data/test.txt", header=None, sep=" ")


# Drop last column
test = test.drop(138, axis=1)

# Drop rows with any NaN values
test.dropna(inplace=True)

print(test.shape)

# Count total NaNs in the DataFrame
test_total_nans = test.isna().sum().sum()
print(test_total_nans)

test_df = df_transform(test)

(26836, 138)
0


<ipython-input-5-54b665ed11d8>:10: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df[df.columns[2:]] = df[df.columns[2:]].applymap(extract_val)


In [10]:
train_df.head(5)

,0,1,2,3,4,5,6,7,8,9,...,128,129,130,131,132,133,134,135,136,137
0,1,3613,4,0,2,0,4,1,0,0.50000,...,101,18,0,256,10674,3,4,0,0,0
1,0,3613,4,0,4,0,4,1,0,1,...,55,5,2,209,15137,2,3,0,0,0
2,0,3613,4,0,2,0,4,1,0,0.50000,...,72,6,0,154,6768,1,4,0,0,0
3,1,3613,4,0,2,0,4,1,0,0.50000,...,101,7,0,222,10707,2,4,0,0,0
4,0,3613,4,0,3,0,4,1,0,0.75000,...,43,227,2,222,13030,1,4,0,0,0


In [12]:
test_df.head(5)

,0,1,2,3,4,5,6,7,8,9,...,128,129,130,131,132,133,134,135,136,137
0,2,13,2,0,2,1,2,1,0,1,...,35,1,0,266,25070,28,7,0,0,0
1,1,13,2,0,0,0,2,1,0,0,...,17,93,0,153,12860,65,158,0,0,0
2,3,13,2,0,1,0,2,1,0,0.50000,...,19,0,0,153,1131,112,141,0,0,0
3,1,13,2,0,2,1,2,1,0,1,...,50,81775,0,560,61224,1,14,0,0,0
4,0,13,1,0,0,0,1,0.50000,0,0,...,24,0,0,57953,15600,15,12,0,0,0


In [17]:
X = train_df[train_df.columns[2:]]
y = train_df[0]
reg = LinearRegression().fit(X, y)

X_test = test_df[test_df.columns[2:]]
y_test = test_df[0]

preds = reg.predict(X_test)

## Mean Squared Error

Mean squared error (MSE) is often used to comapare the difference between true numeric values and predicted values.

In the case of the MSLR Web-10K data set, each query document pair has a true relevance score between 0 and four, and we can use a model to obtain a predicted relevance score. As such, mean squared error can be computed as the mean of, across document-query pairs in the testing set.

We use the mean squared error function from the sci-kit learn module:

In [18]:
from sklearn.metrics import mean_squared_error

mean_squared_error(preds, y_test)

0.5802039232855507

`If we think about a user of a search engine, they don't ever see that predicted value. They are instead presented with an ordered set of documents relating to their query. As such, mean squared error, which only takes into account the numeric values, is perhaps not the best distance metric for learning to rank.`


## Pairwise difference

Instead of considering the predicted and true numeric values associated with the query document pairs, pairwise difference considers only the ordering of the documents. Pariwise difference counts the number of "swaps" of neighbouring documents that have to be made to go from the predicted ordering to the true optimal ordering. The orderings are determined by ranking according to the predicted and true numeric values.

Pairwise difference is computed for each query individually. The mean pariwise difference across all queries can then be computed and used to compare models.

To compute the pairwsie difference, we use the bubble sort algorithm to transform the documents from the predicted order to the true optimal order, and count the nubmer of swaps implemented in the algorithm.

In order to determine the Pairwise Difference we work with each of the queries individually.

Once the data is split by query we need to



In [19]:
preds_df = pd.DataFrame({'qid': test_df[1], 'truth' :test_df[0], 'predicted' : preds })

In [20]:
preds_df.head(5)

,qid,truth,predicted
0,13,2,0.623379
1,13,1,0.255185
2,13,3,0.378686
3,13,1,0.494588
4,13,0,0.432892


In [21]:
groups = preds_df.groupby('qid')

In [22]:
print(groups.size().min())
print(groups.size().max())

4
370


This shows that, for atleast one query, there is only 4 document for which predictions were made. We can remove the contribution of the queries with one doc result to the metric, or we can just leave them since they have the same effect on the metric of any odel considered. For now we keep them.

Let's work with a specific query to compute the pairwise difference

In [23]:
qid_13 = preds_df[preds_df.qid =='13']
qid_13.head(4)

,qid,truth,predicted
0,13,2,0.623379
1,13,1,0.255185
2,13,3,0.378686
3,13,1,0.494588


In [24]:
qid_13.shape

(138, 3)

The first thing we want to do is sort the list by "predicted".

In [25]:
qid_13.sort_values(by = "predicted").head()

,qid,truth,predicted
20,13,0,0.053709
129,13,0,0.235243
1,13,1,0.255185
81,13,0,0.288373
6,13,1,0.297244


We now want to count how many pairwise swaps have to be done by adjacently ordered. To do this we can consider simply the list of true values, in the order sorted by prediction like above.

In [50]:
truth_by_pred = qid_13.sort_values(by = "predicted").truth.tolist()

In [51]:
truth_by_pred[:5]

[0, 0, 1, 0, 1]

In [52]:
def bubble_sort(vals_list):
    swaps = 0
    n=len(vals_list)
    sorted = 0
    while sorted == 0:
        swaps_this_pass = 0
        for i in range(0, n-1):

            if vals_list[i]>vals_list[i+1]:
                vals_list[i], vals_list[i+1] = vals_list[i+1], vals_list[i]
                swaps_this_pass = swaps_this_pass + 1

        if swaps_this_pass==0:
            sorted=1
        swaps = swaps + swaps_this_pass
        n = n-1 #ith pass of bubble sort puts the nth value in order.
            #so there is no need to consider this.
    return(swaps)

In [53]:
bubble_sort(truth_by_pred)

2704

In [54]:
bubble_sort([4, 1, 2, 3])

3

Now we want to write a function which computes this number of pairwise swaps for each query!

In [32]:
def bubble_swaps(group):
    truth_by_pred = group.sort_values(by = "predicted").truth.tolist()
    swaps = bubble_sort(truth_by_pred)
    return swaps

In [33]:
pairwise = groups.apply(bubble_swaps)

<ipython-input-33-1a9d183cb5df>:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  pairwise = groups.apply(bubble_swaps)


In [34]:
pairwise.sample(10)

,0
qid,
3088,198
1768,6239
2368,998
2338,2558
1798,695
1678,348
163,1965
1693,1310
2533,107


In [35]:
pairwise.mean()

1488.5394190871368

### DCG

There are two aspects of ranking which are not caputed by the pairwise difference metric:

`we care about ordering at the start of the document list more than at the end`

`we care about ordering of documents less when the true values are similar.`

`Pairwise difference weights the contribution of every "swap" equally in the distance metric: An out of order pair of documents the end of the list contribute the same amount of weight to the distance metric as an out of order pair at the start.`

`However, if we think about a user of a search engine, they care much more about the order of the list of documents on the first page of search results than they do about the order on the last page.`

`Discounted Cumulative Gain, or dcg, downweights the contributions of documents to the distance metric as you move down the list logarithmically, thereby accounting for the fact that we care more about the start of the list than the end.`

`If we consider the following two lists of three documents. Their true values are: {10,7,9} and {10,8,9}, their pairwise difference from the truth is 1 for both lists, since we must swap the second and third document to get the either list to optimal order. However, ideally we would want to penalise 10,7,9 more than 10,8,9.`

`The metric that fits all our needs is discounted cumulative gain, or dcg.`

In [40]:
import math

def dcg(ordered_data):
    """discounted cumulative gain"""
    n = len(ordered_data)
    if sum(ordered_data)==0:
        return 0
    else:
        indexloop = range(0, n)
        DCG = 0
        for index in indexloop:
            current_ratio=(2**(ordered_data[index])-1)*(math.log((float(index)+2), 2)**(-1))
            DCG = DCG + current_ratio
        return(DCG)

In [41]:
dcg(truth_by_pred) #compute for one query

30.261924410467387

In [42]:
def sort_dcg(unordered_group):
    ordered_group = unordered_group.sort_values(by = "predicted").truth.tolist()
    return dcg(ordered_group)

In [43]:
dcg_score = groups.apply(sort_dcg)

<ipython-input-43-3fe1cd886c60>:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  dcg_score = groups.apply(sort_dcg)


In [44]:
dcg_score.sample(10)

,0
qid,
2083,25.610025
1768,36.362639
883,33.812617
58,15.402076
823,2.572902
1033,16.121614
853,13.259498
448,6.823980
3268,15.798187


`Although this distance metric has a couple of features we are happy with, it's hard to compare the numerical values across queries, since they all have different numbers of documents in them.`

`Normalised discounted cumulative gain, or ndcg,`

In [45]:
""" this returns 0 if all of the ordered data is undesirable"""
def ndcg_p(ordered_data, p):
    """normalised discounted cumulative gain"""
    if sum(ordered_data)==0:
        return 0
    else:
        indexloop = range(0, p)
        DCG_p = 0
        for index in indexloop:
            current_ratio=(2**(ordered_data[index])-1)*(math.log((float(index)+2), 2)**(-1))
            DCG_p = DCG_p + current_ratio
        sorted_data= sorted(ordered_data,reverse=True)
        n = len(ordered_data)
        indexloop = range(0, n)
        iDCG_p = 0
        for index in indexloop:
            current_ratio=(2**(sorted_data[index])-1)*((math.log((index+2), 2))**(-1))
            iDCG_p = iDCG_p + current_ratio
        return(DCG_p/iDCG_p)

In [46]:
truth_by_pred = qid_13.sort_values(by = "predicted").truth.tolist()

In [48]:
try_ndgc=ndcg_p(truth_by_pred, 120)
try_ndgc


0.5498428776642906

In [49]:
truth_by_pred[:5]

[0, 0, 1, 0, 1]

In [55]:
def compute_at_p(ordered_data, p):
    """
    returns -1 if number of items in list is less than p
    else returns ndcg@p
    """
    n = len(ordered_data)
    if n<p:
        result = np.nan
    else:
        result = ndcg_p(ordered_data, p)

    return result

In [56]:
def sort_ndcg(unordered_group, p):
    ordered_group = unordered_group.sort_values(by = "predicted").truth.tolist()
    return compute_at_p(ordered_group, p)

In [57]:
sort_ndcg(qid_13, 120)

0.5498428776642906

In [58]:
compute_at_p(truth_by_pred, 100)

0.16054789989686172

In [59]:
ndcg_score = groups.apply(sort_ndcg, p=100)

<ipython-input-59-e4a81b81f68b>:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  ndcg_score = groups.apply(sort_ndcg, p=100)


In [60]:
ndcg_score.sample(10)

,0
qid,
928,NaN
2038,0.382532
2488,0.340410
1183,NaN
1408,NaN
373,NaN
2323,0.281228
238,0.410898
3298,0.296323


NaN represents that there was not enough documents considered for that query to compute the ndcg at the requested level, p.

In order to summaries all these numbers into one we can use the np.nanmean() function, which ignores any NaN entries when computing the mean:

In [61]:
np.nanmean(ndcg_score)

0.3469803115288955